In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [39]:
data_2018 = pd.read_csv('data_2018.csv', delimiter=";", encoding='latin1')

In [42]:
departamentos = np.array(list(dict.fromkeys(data_2018['Depto'])))
partidos = np.array(data_2018.keys()[8:25])

In [43]:
votos_2018 = {}
for i in departamentos:
    votos_2018[i] = {}
    for j in partidos:
        votos_2018[i][j] = np.sum(data_2018[data_2018['Depto'] == i][j])

In [34]:
data_2022 = pd.read_csv('data_2022.csv', encoding='UTF-8')